In [1]:
# Extract keyords and important topics
# Wordcloud to show the ones that occur the most often
# Visualizations between employee title, tenure, department and the rating they gave

In [2]:
# Heatmap on a map of the world for grades by location

In [3]:
# Extract the top N positives and negative points that come up in reviews

In [4]:
# General sentiment about the company - different segments of the employee population.

# > What are the segments of people we are interested in? Everyone? Mangers and up?
# > What do we want to know about them? How they feel about the company
# > Why do they feel that way? - Topic Extraction and Clustering
# > How has this changed through time? Were people giving better reviews in 2018?

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [33]:
import plotly as py
import cufflinks as cf

In [34]:
from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

### Data import

In [35]:
df = pd.read_csv('Data.csv', index_col = 0)
df.head()

,Title,Rating,Designation,Pros,Cons,Author Info
0,Great Culture and Company That Cares,4,Current Employee,Not much politics and very few ego's. Everyone...,Need to focus on career growth and promoting w...,13 Jan 2021 - Senior Corporate Recruiter in Tr...
1,Shaping the Future of Mobility!,5,"Current Employee, more than 1 year","Global Work Environment, Cutting Edge Technolo...",None to mention at this time.,"14 Apr 2021 - Program Manager in Dublin, Dublin"
2,Lots of potential- getting there,4,"Current Employee, more than 1 year",A very fast paced environment - you definitely...,Training and coaching could be better- high ex...,"22 May 2021 - SCM in Dublin, Dublin"
3,Alright but not the best,3,"Current Employee, more than 1 year","Excellent office, good staff, good pay, good b...","Dis organised, office politics at mid/top leve...","8 Mar 2021 - Business Analyst in Dublin, Dublin"
4,Aptiv,3,Current Employee,It is one of the leaders in the automotive ind...,"High pressure for hitting numbers, it is more ...","7 Nov 2020 - Supply Chain in Dublin, Dublin"


In [30]:
all_data['Date Posted'] = all_data['Author Info'].str[0.8]
all_data['Date Posted'] = all_data['Author Info']
all_data.head(20)

,Title,Rating,Designation,Pros,Cons,Author Info,Date Posted
0,Great Culture and Company That Cares,4,Current Employee,Not much politics and very few ego's. Everyone...,Need to focus on career growth and promoting w...,13 Jan 2021 - Senior Corporate Recruiter in Tr...,13 Jan 2021 - Senior Corporate Recruiter in Tr...
1,Shaping the Future of Mobility!,5,"Current Employee, more than 1 year","Global Work Environment, Cutting Edge Technolo...",None to mention at this time.,"14 Apr 2021 - Program Manager in Dublin, Dublin","14 Apr 2021 - Program Manager in Dublin, Dublin"
2,Lots of potential- getting there,4,"Current Employee, more than 1 year",A very fast paced environment - you definitely...,Training and coaching could be better- high ex...,"22 May 2021 - SCM in Dublin, Dublin","22 May 2021 - SCM in Dublin, Dublin"
3,Alright but not the best,3,"Current Employee, more than 1 year","Excellent office, good staff, good pay, good b...","Dis organised, office politics at mid/top leve...","8 Mar 2021 - Business Analyst in Dublin, Dublin","8 Mar 2021 - Business Analyst in Dublin, Dublin"
4,Aptiv,3,Current Employee,It is one of the leaders in the automotive ind...,"High pressure for hitting numbers, it is more ...","7 Nov 2020 - Supply Chain in Dublin, Dublin","7 Nov 2020 - Supply Chain in Dublin, Dublin"
5,Black stain on the IDA - Kevin and Joe Show - ...,1,"Current Employee, less than 1 year",The marketing department is great at creating ...,Everything about this place is a con. Dont wor...,"11 May 2020 - Financial Analyst in Dublin, Dublin","11 May 2020 - Financial Analyst in Dublin, Dublin"
6,"Bad culture, no HR alignment",1,"Current Employee, more than 3 years","Fairly okay salary, key stakeholders good to w...","HR is all over the place, no centralised HR, g...",13 Jan 2021 - Human Resources Business Partner...,13 Jan 2021 - Human Resources Business Partner...
7,Good working place,4,Former Employee,good package comparing to other companies,IT chaos in terms of org and process,"7 Nov 2020 - Solutions Architect in Dublin, Du...","7 Nov 2020 - Solutions Architect in Dublin, Du..."
8,Very good company advancing the technology for...,5,Current Employee,#NAME?,#NAME?,"10 Aug 2020 - EMEA Internal Audit, Sox & Compl...","10 Aug 2020 - EMEA Internal Audit, Sox & Compl..."
9,Soulless company that grinds you into dust,1,"Current Employee, more than 1 year","Beautiful offices, friendly work colleagues, f...",This is the hardest company I've ever worked f...,"28 Aug 2020 - Corporate Ops in Dublin, Dublin","28 Aug 2020 - Corporate Ops in Dublin, Dublin"


In [36]:
# DROP UNWANTED COLUMNS
'''
df.drop(labels=['column name', 'column name'], axis=1, inplace=True)
df.head()
'''

"\ndf.drop(labels=['column name', 'column name'], axis=1, inplace=True)\ndf.head()\n"

In [39]:
df.isnull().sum()

Title          0
Rating         0
Designation    0
Pros           0
Cons           0
Author Info    0
dtype: int64